##### Copyright 2022 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# RunInference with Sentenced T5 Model from TensorFlow Hub


In this notebook, we walk through the use of the RunInference transform with a [sentence encoder built on T5 model](https://tfhub.dev/google/sentence-t5/st5-base/1) and testing it locally with Interactive Runner.


## Install and import packages.

In [ ]:
!pip install apache_beam[gcp]==2.41.0
!pip install apache-beam[interactive]==2.41.0
!pip install tensorflow==2.10.0
!pip install tensorflow_text==2.10.0
!pip install keras==2.10.0
!pip install tfx_bsl==1.10.0
!pip install pillow==8.4.0

In [ ]:
import os
import importlib

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from tensorflow import keras

from typing import Any
from typing import Dict
from typing import Iterable
from typing import Optional
from typing import Sequence

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.base import ModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.internal import pickler
from apache_beam.runners.runner import PipelineResult
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

from tfx_bsl.public.beam.run_inference import CreateModelHandler
from tfx_bsl.public.proto import model_spec_pb2

## Create a Keras Model from TensorFlow Hub Image

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
PROJECT_ID = '<Project Id>'
GCS_BUCKET = '<GCS Bucket>'

MODEL_PATH = f'{GCS_BUCKET}/st5-base/1'

In [ ]:
inp = tf.keras.layers.Input(shape=[], dtype=tf.string, name='input')
hub_url = "https://tfhub.dev/google/sentence-t5/st5-base/1"
imported = hub.KerasLayer(hub_url)
outp = imported(inp)
model = tf.keras.Model(inp, outp)

# Sentenced-T5 model returns a 768-dimensional vector for an English text input.
# Note the 'input' that we will pass in as example's feature key name.
model.summary()

## Save the model with a TF function definition for RunInference()

In [ ]:
RAW_DATA_PREDICT_SPEC = {
    'input': tf.io.FixedLenFeature([], tf.string),
}

@tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
def call(serialized_examples):
    features = tf.io.parse_example(serialized_examples, RAW_DATA_PREDICT_SPEC)
    return model(features)

tf.saved_model.save(model, MODEL_PATH, signatures={'serving_default': call})

## Create and test a RunInference pipeline locally

In [ ]:
# Creates TensorFlow Example to feed to the ModelHandler.
class ExampleProcessor:
    def create_example(self, feature: tf.string):
        return tf.train.Example(
            features=tf.train.Features(
                  feature={'input' : self.create_feature(feature)})
            )

    def create_feature(self, element: tf.string):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[element.encode()], ))


In [ ]:
saved_model_spec = model_spec_pb2.SavedModelSpec(model_path=MODEL_PATH)
inferece_spec_type = model_spec_pb2.InferenceSpecType(saved_model_spec=saved_model_spec)
model_handler = CreateModelHandler(inferece_spec_type)

questions = [
    'what is the official slogan for the 2018 winter olympics?',
]

pipeline = beam.Pipeline(InteractiveRunner())

inference = (pipeline | 'CreateSentences' >> beam.Create(questions)
               | 'Convert input to Tensor' >> beam.Map(lambda x: ExampleProcessor().create_example(x))
               | 'RunInference with T5' >> RunInference(model_handler))

In [ ]:
ib.show(inference)